In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import importlib
import os
from glob import glob
from tqdm import tqdm
import numpy as np

import datetime

from  utility_programs.read_routines import SAMI

import cartopy.crs as ccrs

from datetime import datetime
import pyinterp


import cartopy.crs as ccrs

In [2]:
sami_data_path = '/glade/u/home/abukowski/scratch/simstorm-20110521/GITM-simstorm-run1/sami-gitm-coupled-lores/'

In [3]:
SAMI = importlib.reload(SAMI)

In [4]:
nz, nf, nlt, nt = SAMI.get_grid_elems_from_parammod(sami_data_path)

old_shape = [nlt, nf, nz]

grid2 = SAMI.get_sami_grid(sami_data_path, nlt, nf, nz)

data2, times = SAMI.read_to_nparray(sami_data_path, datetime(2011,5,20),
                                    datetime(2011,5,21,12), 1, 2, cols='edens')

grid = {}

for k in grid2.keys():
    grid[k] = grid2[k].flatten()
    # grid2[k] = grid2[k]
    print(k, grid[k].shape, grid2[k].shape)
    


skipping deni1u.dat because it is not in cols
skipping deni2u.dat because it is not in cols
skipping deni3u.dat because it is not in cols
skipping deni4u.dat because it is not in cols
skipping deni5u.dat because it is not in cols
skipping deni6u.dat because it is not in cols
skipping deni7u.dat because it is not in cols
skipping denn1u.dat because it is not in cols
skipping denn2u.dat because it is not in cols
skipping denn3u.dat because it is not in cols
skipping denn4u.dat because it is not in cols
skipping denn5u.dat because it is not in cols
skipping denn6u.dat because it is not in cols
skipping denn7u.dat because it is not in cols
skipping teu.dat because it is not in cols
skipping ti1u.dat because it is not in cols
skipping ti2u.dat because it is not in cols
skipping ti5u.dat because it is not in cols
skipping vsi1u.dat because it is not in cols
skipping vsi2u.dat because it is not in cols
skipping u1pu.dat because it is not in cols
skipping u3hu.dat because it is not in cols
ski

In [5]:
mesh = pyinterp.RTree()

mesh.packing(np.vstack((grid['glon'], grid['glat'], grid['alt'])).T, data2['data']['edens'][:,:,:,5].flatten())

In [6]:
latout = np.arange(-90,90,1)
lonout = np.arange(0,360,4)
altout = [450] #np.arange(400,1000,50)

mx, my, mz = np.meshgrid(lonout,latout,altout,
                        indexing='ij')

In [ ]:
rbf, neighbors = mesh.radial_basis_function(
    np.vstack((mx.ravel(), my.ravel(), mz.ravel())).T,
    within=False,  # Extrapolation is forbidden
    k=1600,  # We are looking for at most 11 neighbors
    rbf='linear',
    smooth=1e-4,
    num_threads=2)
rbf = rbf.reshape(mx.shape)

In [ ]:
rbf.shape

In [ ]:
plt.imshow(rbf[:,:,5])